In [8]:

#r "nuget: Microsoft.SemanticKernel, 0.17.230629.1-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 0.17.230629.1-preview"
#r "nuget: Microsoft.SemanticKernel.Core, 0.17.230629.1-preview"
#r "nuget: Microsoft.SemanticKernel.Planning.StepwisePlanner, 0.17.230629.1-preview"

Installed Packages Microsoft.SemanticKernel, 0.17.230629.1-preview Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 0.17.230629.1-preview microsoft.semantickernel.core, 0.17.230629.1-preview Microsoft.SemanticKernel.Planning.StepwisePlanner, 0.17.230629.1-preview

In [20]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;

IKernel semanticKernel = Kernel.Builder
                    .WithOpenAITextCompletionService(
                        "text-davinci-003",                     // OpenAI Model Name
                        "sk-" // OpenAI Key
                    )
                    .Build();

# Skill 1: Register Native Function

I am copying directily from https://github.com/microsoft/semantic-kernel two basic native function

In [21]:
// Copyright (c) Microsoft. All rights reserved.

using System.ComponentModel;
using Microsoft.SemanticKernel.SkillDefinition;

public sealed class StaticTextSkill
{
    [SKFunction, Description("Change all string chars to uppercase")]
    public static string Uppercase([Description("Text to uppercase")] string input) =>
        input.ToUpperInvariant();

    [SKFunction, Description("Append the day variable")]
    public static string AppendDay(
        [Description("Text to append to")] string input,
        [Description("Value of the day to append")] string day) =>
        input + day;

    [SKFunction, Description("Get today's day")]
    public static string GetTodayDay() =>
        DateTime.Today.DayOfWeek.ToString();
}

// register native function
semanticKernel.ImportSkill(new StaticTextSkill(), "StaticTextSkill");

# Define PLanner

In [22]:
#r "nuget: Microsoft.SemanticKernel.Planning.SequentialPlanner, 0.17.230629.1-preview"

Installed Packages Microsoft.SemanticKernel.Planning.SequentialPlanner, 0.17.230629.1-preview

In [23]:
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planning.Sequential;

var plannerConfig = new SequentialPlannerConfig();
plannerConfig.MaxTokens = 1000;

SequentialPlanner planner = new SequentialPlanner(semanticKernel, plannerConfig);


In [24]:
var plan = await planner.CreatePlanAsync("convert this text to upper case and append today's day : Hi, today is lovely day :  ");

In [25]:
var result = await plan.InvokeAsync(semanticKernel.CreateNewContext());


In [29]:
Console.WriteLine("\nPlan: " + plan.ToJson());
Console.WriteLine("\n\nFinal Answer is : " + result);


Plan: {"state":[{"Key":"RESULT__FINAL_ANSWER","Value":"HI, TODAY IS LOVELY DAY :SATURDAY"},{"Key":"PLAN.RESULT","Value":"\nHI, TODAY IS LOVELY DAY :SATURDAY"},{"Key":"TODAY_DAY","Value":"Saturday"},{"Key":"INPUT_WITH_DAY","Value":"Hi, today is lovely day :Saturday"},{"Key":"INPUT","Value":"HI, TODAY IS LOVELY DAY :SATURDAY"}],"steps":[{"state":[{"Key":"INPUT","Value":""}],"steps":[],"parameters":[{"Key":"INPUT","Value":""}],"outputs":["TODAY_DAY"],"next_step_index":0,"name":"GetTodayDay","skill_name":"StaticTextSkill","description":"Get today\u0027s day"},{"state":[{"Key":"INPUT","Value":""}],"steps":[],"parameters":[{"Key":"day","Value":"$TODAY_DAY"},{"Key":"INPUT","Value":"Hi, today is lovely day :"}],"outputs":["INPUT_WITH_DAY"],"next_step_index":0,"name":"AppendDay","skill_name":"StaticTextSkill","description":"Append the day variable"},{"state":[{"Key":"INPUT","Value":""}],"steps":[],"parameters":[{"Key":"INPUT","Value":"$INPUT_WITH_DAY"}],"outputs":["RESULT__FINAL_ANSWER"],"next